In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
import torch.nn.functional as f

In [ ]:
# Load MNIST data
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

batch_size = 100
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 14607240.16it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 483764.88it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4448167.93it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 7694074.62it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Hyper parameters
learning_rate = 1e-2
batch_size = 100
num_classes = 10
num_epochs = 10

In [ ]:
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)

        self.fc1 = nn.Linear(in_features=16 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(f.relu(self.conv1(x)))
        x = self.pool(f.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = f.relu(self.fc1(x))
        x = f.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 300 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_steps}], Loss: {loss.item():.4f}')
print('Done')

Epoch [1/10], Step [300/600], Loss: 2.2960
Epoch [1/10], Step [600/600], Loss: 2.2665
Epoch [2/10], Step [300/600], Loss: 1.5423
Epoch [2/10], Step [600/600], Loss: 0.6707
Epoch [3/10], Step [300/600], Loss: 0.3748
Epoch [3/10], Step [600/600], Loss: 0.2098
Epoch [4/10], Step [300/600], Loss: 0.1402
Epoch [4/10], Step [600/600], Loss: 0.3426
Epoch [5/10], Step [300/600], Loss: 0.2523
Epoch [5/10], Step [600/600], Loss: 0.2025
Epoch [6/10], Step [300/600], Loss: 0.2215
Epoch [6/10], Step [600/600], Loss: 0.0919
Epoch [7/10], Step [300/600], Loss: 0.1010
Epoch [7/10], Step [600/600], Loss: 0.0950
Epoch [8/10], Step [300/600], Loss: 0.0710
Epoch [8/10], Step [600/600], Loss: 0.1044
Epoch [9/10], Step [300/600], Loss: 0.1601
Epoch [9/10], Step [600/600], Loss: 0.1338
Epoch [10/10], Step [300/600], Loss: 0.1244
Epoch [10/10], Step [600/600], Loss: 0.0784
Done


In [ ]:
with torch.no_grad():
    correct = 0
    total = 0

    for images, labels in test_loader:

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = correct / total
    print(f'Accuracy: {100 * acc:.2f}%')

Accuracy: 96.91%
